In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from tqdm.notebook import tqdm

In [2]:
NEURONS_COUNT = 10
EPOCH = 10
LEARNING_SPEED = 0.1
THETA = 0.5
MSE_LOWER_LIMIT = 1e-16
MSE_LOWER_DIFFERENCE = 1e-15

error = 0

In [3]:
(data_train, label_train), (data_test, label_test) = mnist.load_data()

In [4]:
data_train = np.array([data.flatten() / 255 for data in data_train])
data_test = np.array([data.flatten() / 255 for data in data_test])

In [5]:
train_length = len(data_train)
test_length = len(data_test)

In [6]:
class Neuron:
    _SIZE_OF_WEIGHTS = 784


    def __init__(self, label: int, learning_speed = 0.01) -> None:
        self._weights = np.array(np.random.uniform(-0.03, 0.03, self._SIZE_OF_WEIGHTS))
        self._bias = np.random.rand()
        self._label = label
        self._learning_speed = learning_speed
        self._mse = 0


    def __str__(self) -> str:
        return str(self._label)


    def train(self, inputed_value: np.ndarray, inputed_label: np.ndarray):
        supposed_result = int(inputed_label == self._label)
        weighted_sum = self._get_weighted_sum(inputed_value)
        sigmoid = self._get_sigmoid(weighted_sum)

        self._mse = self._get_mse(sigmoid, supposed_result)        
        
        self._weights, self._bias = self._get_corrected_values(inputed_value, supposed_result, sigmoid)

    
    def get_match_value(self, inputed_value: np.ndarray) -> float:
        return self._get_sigmoid(self._get_weighted_sum(inputed_value))


    def get_local_error(self) -> float:
        return self._mse


    def _get_sigmoid(self, weighted_sum: np.ndarray) -> float:
        return 1 / (1 + np.exp(-weighted_sum))
    

    def _get_sigmoid_diff(self, sigmoid: float) -> float:
        return sigmoid * (1 - sigmoid)
    

    def _get_learning_speed_coefficient(self, sigmoid: float, supposed_result: float) -> float:
        return -(supposed_result - sigmoid) * self._get_sigmoid_diff(sigmoid)
    

    def _get_correction(self, sigmoid: float, supposed_result: float) -> float:
        return -self._learning_speed * self._get_learning_speed_coefficient(sigmoid, supposed_result)
    

    def _get_corrected_values(self, inputed_value: np.ndarray, supposed_result: float, sigmoid: float) -> np.ndarray:
        correction = self._get_correction(sigmoid, supposed_result)
        return (self._weights + correction * inputed_value, self._bias + correction)
    
    
    def _get_weighted_sum(self, inputed_value: np.ndarray) -> float:
        return np.dot(self._weights, inputed_value) + self._bias
    

    def _get_mse(self, current_result: float, supposed_result: float) -> float:
        return (supposed_result - current_result) * (supposed_result - current_result) / 2

In [15]:
def get_neurons(count: int):
    return [Neuron(i, LEARNING_SPEED) for i in range(count)]

In [17]:
def train(train_data, train_label, neurons):
    success_count: float = 0
    total_count: float = 0
    mse: float = 0

    previous_mse_epoch = 0
    for _ in tqdm(range(EPOCH), 'Эпохи'):
        previous_mse = 0
        mse_epoch = 0

        for (data, label) in tqdm(zip(train_data, train_label), 'Итерации', total=train_length):
            mse = 0

            for neuron in neurons:
                result = neuron.train(data, label)

                mse += neuron.get_local_error()

                success_count += int((str(label) == str(neuron)) and neuron.get_match_value(data) >= 0.5)
                total_count += 1

            mse /= NEURONS_COUNT
            mse_epoch += mse
            
            if mse < MSE_LOWER_LIMIT:
                return (success_count / total_count, mse)
            
            previous_mse = mse

        mse_epoch /= train_length

        if abs(mse_epoch - previous_mse_epoch) < MSE_LOWER_DIFFERENCE:
            return (success_count / total_count, mse)
            

    return (success_count / total_count, mse)

In [18]:
def test(test_data, test_label, neurons):
    counter = 0
    for data, label in zip(test_data, test_label):
        biggest_match = 0
        neuron_label = -1

        for neuron in neurons:
            current_match = neuron.get_match_value(data)

            if current_match > biggest_match:
                biggest_match = current_match
                neuron_label = int(str(neuron))

        if neuron_label == label and biggest_match > 0.5:
            counter += 1

    return counter / test_length

In [21]:
neurons = get_neurons(NEURONS_COUNT)

In [22]:
print(train(data_train, label_train, neurons))

Эпохи:   0%|          | 0/10 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

Итерации:   0%|          | 0/60000 [00:00<?, ?it/s]

(0.09038453475208363, 5.923090310472955e-17)


In [23]:
test(data_test, label_test, neurons)

0.8601